# Persistence and Efficency
You now might be aware of some of the problems with parsing several pages and how that can quicly get out of hand. While parsing, you might chose to persist the collected data, so that it can be analyzed and cleaned later. 
Parsing, retrieving, saving, and cleaning data are all separate actions, and you shouldn't try to work with data while collecting. In this notebook you'll practice further parsing techniques along with persistency, both for JSON and CSV formats as well as using SQL and a database.

Start by loading one of the available HTML files into the `scrapy` library

**Note**: This notebook relies on several dependencies that must be pre-installed in your environment. If you haven't done so, please run the following command in your terminal or command prompt:
```bash
python3 -m venv venv
venv/bin/pip install -r requirements.txt
```

In [9]:
import scrapy
import os
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "html/1994_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url) as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')

In [10]:
# Make sure that the interesting data is available 
table = response.xpath('//table')[1].xpath('tbody')
for tr in table.xpath('tr'):
    print(tr.xpath('td/b/text()').extract()[0],
          tr.xpath('td/a/text()').extract()[0]
    )

Gold Jagan Hames
Silver Antoine Burke
Bronze Mika Polku


This interaction with `scrapy` in a Jupyter Notebook is useful because you don't need to run the special shell and you also don't need to run the whole spider. Once you learn what you need to do here, you can adapat the spider to persist data.
First, start by persisting data as JSON. To do this, you will need to keep the information in a Python data structure like a dictionary, and then load it as a JSON object, and finally, save it to a file.

In [11]:
scrapped_data = {}
for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    scrapped_data[medal] = athlete

scrapped_data

{'Gold': 'Jagan Hames', 'Silver': 'Antoine Burke', 'Bronze': 'Mika Polku'}

In [12]:
import json

# You can convert Python into JSON first, but there is no need if you use `json.dump()`
# as shown next
json_data = json.dumps(scrapped_data)

# Persist it in a file:
with open("1994_results.json", "w") as _f:
    # use dump() with the Python dictionary directly. 
    # the conversion is done on the fly
    json.dump(scrapped_data, _f)

Now that you can persist the scrapped data as JSON, you can also use CSV. This is specially useful if you want to to some data science operations. Although you can use an advanced library like Pandas for this, you can use the standard library CSV module from Python.

In [13]:
# construct the data first

column_names = ["Medal", "Athlete"]
rows = []

for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    rows.append([medal, athlete])


In [14]:
# Now persist it to disk
import csv

with open("1994_results.csv", "w") as _f:
    writer = csv.writer(_f)

    # write the column names
    writer.writerow(column_names)

    # now write the rows
    writer.writerows(rows)

Finally, you can persist data to a database. Unlike the JSON and CSV approach, using a database is much more memory efficient. This is the principal reason why you want to use a database instead of a file on disk. Imagine capturing 10GB of data. This could potentially mean that you need 10GB of available memory to hold onto that data before saving it to disk.
By using a database, you can save the data as the data is gathered. 

For the next cells, use a SQLite database to persist the data. Create the file-based database and the table needed.

In [15]:
import sqlite3
connection = sqlite3.connect("1994_results.db")
db_table = 'CREATE TABLE results (id integer primary key, medal TEXT, athlete TEXT)'
cursor = connection.cursor()
cursor.execute(db_table)
connection.commit()

In [16]:
# Now it is time to persist the data. Open the connection again
connection = sqlite3.connect("1994_results.db")
cursor = connection.cursor()
query = 'INSERT INTO results(medal, athlete) VALUES(?, ?)'

for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    cursor.execute(query, (medal, athlete)) 
    connection.commit()

The data is now persisted in a file-based database that you can query. Verify that all works by creating a new connection and querying the database.

Update the _wikipedia_ project and spider to use some of these techniques to persist data. Next, try parsing all the files in the _html_ directory instead of just one and persist all results. Do you think you can parse other information as well? 

Try parsing the height and the results from all the other athletes, not just the top three places.

In [ ]:
import scrapy
import os
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "html/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
url = os.path.join(current_dir, "html/1994_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
url = os.path.join(current_dir, "html/1996_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
url = os.path.join(current_dir, "html/1998_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url) as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')

In [24]:
import scrapy
import os
import csv
import json
import sqlite3

# Define your file paths here
file_paths = [
    r"C:\Users\oscar\OneDrive\Oscar\Master_Data_Science_AI_Nuclio_School\Projects\Coursera_Webscrapping\1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump",
    r"C:\Users\oscar\OneDrive\Oscar\Master_Data_Science_AI_Nuclio_School\Projects\Coursera_Webscrapping\1994_World_Junior_Championships_in_Athletics_–_Men's_high_jump",
    r"C:\Users\oscar\OneDrive\Oscar\Master_Data_Science_AI_Nuclio_School\Projects\Coursera_Webscrapping\1996_World_Junior_Championships_in_Athletics_–_Men's_high_jump",
    r"C:\Users\oscar\OneDrive\Oscar\Master_Data_Science_AI_Nuclio_School\Projects\Coursera_Webscrapping\1998_World_Junior_Championships_in_Athletics_–_Men's_high_jump"
]

def ordinal(n):
    """Helper function for ordinal numbers."""
    return "%d%s" % (n, "th" if 4 <= n % 100 <= 20 else {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th"))

# List to store all athlete data
athletes_data = []

for file_path in file_paths:
    try:
        with open(file_path, 'r', encoding='utf-8') as _f:
            url_data = _f.read()

        response = scrapy.http.TextResponse(url='file://' + file_path, body=url_data.encode('utf-8'), encoding='utf-8')
        event_name = os.path.basename(file_path)
        print(event_name)

        athlete_rank = 0
        for tr in response.xpath('//table//tr[position()>1]'):
            if athlete_rank >= 10:  # Stop after processing 10 athletes
                break

            athlete = tr.xpath('./td[2]//text()').get(default='').strip()
            nationality = tr.xpath('./td[3]//a/text()').get(default='').strip()
            result = tr.xpath('./td[4]/text()').get(default='').strip() or tr.xpath('./td[4]/b/text()').get(default='').strip()

            if not athlete or athlete.lower() == "women" or not result:
                continue

            athlete_rank += 1
            position = ordinal(athlete_rank)

            athletes_data.append({
                "Event": event_name,
                "Position": position,
                "Athlete": athlete,
                "Nationality": nationality,
                "Result": result
            })

    except FileNotFoundError:
        print(f"File not found for: {event_name}")
    except Exception as e:
        print(f"An error occurred with {event_name}: {e}")

# Writing to CSV
csv_file = "allyearsdata.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Event", "Position", "Athlete", "Nationality", "Result"])
    writer.writeheader()
    for data in athletes_data:
        writer.writerow(data)

# Writing to JSON
json_file = "allyearsdata.json"
with open(json_file, 'w', encoding='utf-8') as file:
    json.dump(athletes_data, file, ensure_ascii=False, indent=4)

# Writing to SQLite DB
db_file = "allyearsdata.db"
conn = sqlite3.connect(db_file)
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS athletes
             (Event TEXT, Position TEXT, Athlete TEXT, Nationality TEXT, Result TEXT)''')
for data in athletes_data:
    c.execute("INSERT INTO athletes VALUES (:Event, :Position, :Athlete, :Nationality, :Result)", data)
conn.commit()
conn.close()

print("Data saved to CSV, JSON, and SQLite DB.")


File not found for: 1998_World_Junior_Championships_in_Athletics_–_Men's_high_jump
File not found for: 1998_World_Junior_Championships_in_Athletics_–_Men's_high_jump
File not found for: 1998_World_Junior_Championships_in_Athletics_–_Men's_high_jump
File not found for: 1998_World_Junior_Championships_in_Athletics_–_Men's_high_jump
Data saved to CSV, JSON, and SQLite DB.


In [25]:
import scrapy
import os

# Get the current directory
current_dir = os.path.abspath('')

# List of event years, assuming you're working with a specific set of files
event_years = ["1992", "1994", "1996", "1998"]

# Base directory where the HTML files are stored
base_dir = os.path.join(current_dir, "html")

# Process each file
for year in event_years:
    # Construct the file name and path
    file_name = f"{year}_World_Junior_Championships_in_Athletics_–_Men's_high_jump"
    file_path = os.path.join(base_dir, file_name)
    
    # Try to open and read the file
    try:
        with open(file_path, 'r', encoding='utf-8') as _f:
            url_data = _f.read()

        # Create a TextResponse object to simulate a response object from Scrapy
        response = scrapy.http.TextResponse(url='file://' + file_path, body=url_data.encode('utf-8'), encoding='utf-8')

        # Now you can process the response object with Scrapy selectors
        # For example, to print the title of each HTML file:
        title = response.xpath('//title/text()').get()
        print(f"Title of {year}: {title}")

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred with {file_path}: {e}")


Title of 1992: 1992 World Junior Championships in Athletics – Men's high jump - Wikipedia
Title of 1994: 1994 World Junior Championships in Athletics – Men's high jump - Wikipedia
Title of 1996: 1996 World Junior Championships in Athletics – Men's high jump - Wikipedia
Title of 1998: 1998 World Junior Championships in Athletics – Men's high jump - Wikipedia


In [29]:
import requests
from parsel import Selector
import os
import csv
import json
import sqlite3

# Define your URLs here
urls = [
    "https://en.wikipedia.org/wiki/1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump",
    "https://en.wikipedia.org/wiki/1994_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump",
    "https://en.wikipedia.org/wiki/1996_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump",
    "https://en.wikipedia.org/wiki/1998_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump"
]

def ordinal(n):
    """Helper function for ordinal numbers."""
    return "%d%s" % (n, "th" if 4 <= n % 100 <= 20 else {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th"))

# List to store all athlete data
athletes_data = []

for url in urls:
    try:
        response = requests.get(url)
        response.raise_for_status()  # This will raise an HTTPError if the response was an error
        
        selector = Selector(response.text)
        event_name = os.path.basename(url)
        print(event_name)

        athlete_rank = 0
        for tr in selector.xpath('//table//tr[position()>1]'):
            if athlete_rank >= 10:  # Stop after processing 10 athletes
                break

            athlete = tr.xpath('./td[2]//text()').get(default='').strip()
            nationality = tr.xpath('./td[3]//a/text()').get(default='').strip()
            result = tr.xpath('./td[4]/text()').get(default='').strip() or tr.xpath('./td[4]/b/text()').get(default='').strip()

            if not athlete or athlete.lower() == "women" or not result:
                continue

            athlete_rank += 1
            position = ordinal(athlete_rank)

            athletes_data.append({
                "Event": event_name,
                "Position": position,
                "Athlete": athlete,
                "Nationality": nationality,
                "Result": result
            })

    except requests.HTTPError as e:
        print(f"HTTP error occurred for {url}: {e}")
    except Exception as e:
        print(f"An error occurred with {url}: {e}")

# Writing to CSV
csv_file = "allyearsdata.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Event", "Position", "Athlete", "Nationality", "Result"])
    writer.writeheader()
    for data in athletes_data:
        writer.writerow(data)

# Writing to JSON
json_file = "allyearsdata.json"
with open(json_file, 'w', encoding='utf-8') as file:
    json.dump(athletes_data, file, ensure_ascii=False, indent=4)

# Writing to SQLite DB
db_file = "allyearsdata.db"
conn = sqlite3.connect(db_file)
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS athletes
             (Event TEXT, Position TEXT, Athlete TEXT, Nationality TEXT, Result TEXT)''')
for data in athletes_data:
    c.execute("INSERT INTO athletes VALUES (:Event, :Position, :Athlete, :Nationality, :Result)", data)
conn.commit()
conn.close()

print("Data saved to CSV, JSON, and SQLite DB.")
print(athletes_data)


1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
1994_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
1996_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
1998_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
Data saved to CSV, JSON, and SQLite DB.
[{'Event': '1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump', 'Position': '1st', 'Athlete': 'Steve Smith', 'Nationality': 'United Kingdom', 'Result': '2.37'}, {'Event': '1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump', 'Position': '2nd', 'Athlete': 'Tim Forsyth', 'Nationality': 'Australia', 'Result': '2.31'}, {'Event': '1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump', 'Position': '3rd', 'Athlete': 'Takahiro Kimino', 'Nationality': 'Japan', 'Result': '2.29'}, {'Event': '1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump', 'Position': '4th', 'Athlete': 'Kim Tae You

TypeError: 'NoneType' object is not callable

In [30]:
# Assuming athletes_data is already populated as per the previous steps

# Function to print each athlete's data in an orderly manner
def print_athletes_data(data):
    for entry in data:
        print(f"Event: {entry['Event']}")
        print(f"Position: {entry['Position']}")
        print(f"Athlete: {entry['Athlete']}")
        print(f"Nationality: {entry['Nationality']}")
        print(f"Result: {entry['Result']}\n")

# Call the function to print data
print_athletes_data(athletes_data)


Event: 1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
Position: 1st
Athlete: Steve Smith
Nationality: United Kingdom
Result: 2.37

Event: 1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
Position: 2nd
Athlete: Tim Forsyth
Nationality: Australia
Result: 2.31

Event: 1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
Position: 3rd
Athlete: Takahiro Kimino
Nationality: Japan
Result: 2.29

Event: 1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
Position: 4th
Athlete: Kim Tae Young
Nationality: South Korea
Result: 2.23

Event: 1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
Position: 5th
Athlete: Sergey Klyugin
Nationality: Commonwealth of Independent States
Result: 2.20

Event: 1992_World_Junior_Championships_in_Athletics_%E2%80%93_Men%27s_high_jump
Position: 6th
Athlete: Kristofer Lamos
Nationality: Germany
Result: 2.20

Event: 1992_World_Junior_Championships_in_Athleti